Данный скрипт предназначен для вычисления ряда акустических параметров пения птиц. На вход подаётся директория, содержащая логи с данными, полученные из приложения Raven Ligth (Selection Tables). На выходе получаются 4 файла: файл с рассчитанными параметрами (parameters) и файл с описательными статистиками данных параметров, оба в двух форматах (csv и xls). Расчитываемые параметры каждого лога: длительность, общее кол-во песен, число типов песен (ЕР) в записи (diversity), медианную длительность песен, медианную длительность пауз между песнями, частоту смены напева, Slin, Scons, RE1, versatility. Язык скрипта - Python 3.9, используемые библиотеки: Pandas, Numpy, Os.

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings('ignore')

In [2]:
# Здесь необходимо прописать рабочую директорию' C:\Users\..

home = r'C:\Users\..' 
os.chdir(home)

In [3]:
data_files = []   
for i in os.listdir(home):
    if 'txt' in i:
        data_files.append(i)

# Ищем файлы с логами в директории. Т.к. скрипт написан под Selection Tables от Raven Ligth, то и поиск идёт по расширению txt.

In [4]:
# Преобразуем найденные файлы в датафреймы и соберём в единый список.

data_files_df = []

for file in data_files:
    df = pd.read_csv(file, sep = '\t')
    name = file.split('.')[0]
    df['name'] = name
    data_files_df.append(df)

In [5]:
def newname(x):
    x = x.lower()
    return x
    # функция для перевода наименования в нижний регистр    

In [6]:
# Фукция для расчёта переходных вероятностей

def transition_matrix(transitions):
    n = 1+ max(transitions) #number of states

    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

In [7]:
parameters = pd.DataFrame() # создадим пустой датафрейм, куда будут добавляться параметры по каждому логу

for df in data_files_df:
    df = df.rename(columns = {'Begin Time (s)':'begin_time', 'End Time (s)':'end_time', 'Low Freq (Hz)':'low_freq', 
                              'High Freq (Hz)':'high_freq', 'Delta Time (s)':'delta_time',  'Delta Freq (Hz)':'delta_freq', 
                              'Avg Power Density (dB FS/Hz)':'power'}).rename(columns = newname)

    total_songs = df.shape[0]


    trans = df.annotation.to_list() # переводим столбец с ЕР в список, дальше итерируемся по нему 
    total = 0
    total_trans = (total_songs-1) # df.shape[0] - кол-во строк, т.е. наблюдений. Всего переходов = кол-во наблюдений-1
    for i in range(len(trans)-1):
        if trans[i] != trans[i+1]:
            total += 1             # переменная total отображает кол-во переходов к ЕР другого типа
    switching_rate = total / total_trans 

    diversity = df.annotation.nunique() # размер репертуара в записи (кол-во разных типов ЕР)

    df1 = df[['selection', 'annotation']] # сохраним новую таблицу для расчётов в отдельной переменной

    df1['annotation2'] = df.annotation.shift(-1) # для подсчёта переходов создадим второй столбец со сдвигом

    df1 = df1.dropna()
    # уберём последнее значение как пропущенное, сохраним новую таблицу в отдельной переменной

    df1.annotation2 = df1.annotation2.astype(int) # поправим тип переменной - уберём нули

    transactions = df1.groupby(['annotation', 'annotation2'], as_index = False).agg({'selection' : 'count'})\
    .pivot(index='annotation', columns='annotation2', values='selection' ).fillna(0) # получаем матрицу переходов

    n = transactions.shape[0] # размерность матрицы
    # если у нас матрица не квадратная (т.е. для первого ЕР нет на него перехода), 
    # то необходимо искусственно добавить лишний столбец к матрице, иначе будет ошибка.
    if transactions.shape[0] != transactions.shape[1]:  
        transactions.insert(0, "0", 0)

    matrix = transactions.to_numpy() # сохраним датафрейм с переходами как матрицу

    count = 0
    for i in range(n):
        for j in range(n):
            if matrix[i][j] != 0:
                count += 1 # вычисляем кол-во типов переходов
    if count != 0:
        SLIN = diversity / count
    else:
        SLIN = 1

    sum_freq_trans = 0

    for l in matrix.tolist():
        sum_freq_trans += max(l)  # ∑ наиболее частых переходов (для каждого типа ЕР)

    SCONS = sum_freq_trans / total_trans
    
    S = (SLIN+SCONS)/2

    E0  = -diversity**2*(1/diversity * np.log2(1/diversity))  # формула Шеннона для E0 

    # Фукция для расчёта переходных вероятностей


    m = transition_matrix(trans)  # Матрица переходных вероятностей

    E1 = 0
    for i in range(n):
        for j in range(n):
            if m[i][j] != 0:
                E1 += m[i][j]*np.log2(m[i][j]) # вычисляем Е1 по формуле Шеннона для каждой вероятности, не равной 0. 
    E1 = -E1  
    if diversity > 1:  # В случае, если у нас всего 1 тип песни, при расчёте RE1 по стандартной формуле мы получим NaN.
        RE1=E1/E0  
    else:
        RE1=-E0   # Поэтому добавляем условие, и если репертуар состоит всего из одной песни, RE1 считаем равным Е1.

    if len(trans) >= 10:
        div = []
        for i in range(0, (len(trans) - len(trans)%10), 10): # создаём цикл от начала до последней десятки последовательности с шагом 10
            div.append(len(set(trans[i:i+10])))
        Vers = np.median(div)
    else:
        Vers = len(set(trans))

    fon_length =  df.end_time.iloc[-1] - df.begin_time.iloc[0] # общая длительность фонограммы

    song_rate = df.shape[0] / (fon_length / 60)  # частота пения
       
    df['pause_length'] = df.begin_time.shift(-1) - df.end_time # добавляем колонку с длительностью пауз

    time_param = df[['delta_time', 'pause_length']].describe() # рассчитваем временнЫе параметры

    median_lenth = time_param.delta_time.loc['50%']
    median_pause = time_param.delta_time.loc['50%']

    param = pd.DataFrame({'name': df.name.iloc[0], 'total_lenght': fon_length.round(2), 'total_songs': total_songs, 
                          'diversity':diversity, 'median_song_lenth':median_lenth.round(2),
                          'median_pause_length':median_pause.round(2), 'song_rate':song_rate.round(2), 
                          'switching_rate':round(switching_rate, 2),
                          'SLIN':round(SLIN, 2), 'SCONS':round(SCONS, 2), 'S':round(S, 2),'RE1':RE1.round(2), 'versatility':Vers}, index=[0])

    parameters = parameters.append(param, ignore_index=True)

In [8]:
parameters # ну посмотрим, что получилось :)

,name,total_lenght,total_songs,diversity,median_song_lenth,median_pause_length,song_rate,switching_rate,SLIN,SCONS,S,RE1,versatility
0,Самец10_123,358.65,10,1,0.89,0.89,1.67,0.00,1.00,1.00,1.00,0.00,1.0
1,Самец10_62,325.31,47,2,0.78,0.78,8.67,0.02,0.67,0.98,0.82,0.25,1.0
2,Самец10_63,150.56,29,2,0.85,0.85,11.56,0.04,0.67,0.96,0.82,0.00,1.5
3,Самец10_64,140.52,18,1,0.84,0.84,7.69,0.00,1.00,1.00,1.00,0.00,1.0
4,Самец12_15,458.10,35,6,0.43,0.43,4.58,0.18,0.60,0.88,0.74,0.15,2.0
5,Самец15_23,44.70,12,7,0.60,0.60,16.11,0.82,0.64,0.64,0.64,0.16,6.0
6,Самец1_109,179.70,18,3,0.62,0.62,6.01,0.18,0.60,0.88,0.74,0.06,2.0
7,Самец1_28,275.37,14,1,1.04,1.04,3.05,0.00,1.00,1.00,1.00,0.00,1.0
8,Самец21_87,185.30,10,2,0.80,0.80,3.24,0.11,0.67,0.89,0.78,0.08,2.0
9,Самец2_37,104.33,21,10,0.90,0.90,12.08,0.85,0.59,0.60,0.59,0.14,6.5


In [9]:
parameters.to_csv('parameters.csv') # сохраняем результаты в папку в двух форматах.
parameters.to_excel('parameters.xlsx')

In [10]:
results = parameters.describe().round(2) # рассчитываем описательные статистики и также сохраняем в файлы
results.to_csv('results.csv')
results.to_excel('results.xlsx')
results

,total_lenght,total_songs,diversity,median_song_lenth,median_pause_length,song_rate,switching_rate,SLIN,SCONS,S,RE1,versatility
count,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00
mean,234.90,23.69,3.38,0.82,0.82,7.29,0.19,0.76,0.89,0.82,0.06,2.09
std,134.20,15.40,3.28,0.27,0.27,3.94,0.31,0.19,0.16,0.16,0.08,1.80
min,44.70,10.00,1.00,0.29,0.29,1.67,0.00,0.46,0.49,0.47,0.00,1.00
25%,131.47,11.75,1.00,0.74,0.74,4.24,0.00,0.63,0.88,0.74,0.00,1.00
50%,207.84,18.00,2.00,0.84,0.84,7.36,0.04,0.67,0.96,0.82,0.00,1.25
75%,360.74,30.50,3.75,0.89,0.89,9.09,0.18,1.00,1.00,1.00,0.11,2.00
max,458.10,56.00,11.00,1.46,1.46,16.11,0.85,1.00,1.00,1.00,0.25,6.50
